In [1]:
from helper_functions import *

In [2]:
pwd

'/Users/jeremymobley/Desktop/cbb_modeling/Python'

In [3]:
cd /Users/jeremymobley/Desktop/cbb_modeling/data/2018/

/Users/jeremymobley/Desktop/cbb_modeling/data/2018


In [4]:
from IPython.display import display, HTML
import pandas as pd
import numpy as np

#### Load Data

In [5]:
teams = read_in_file_to_df("Teams")
tourney_compact_results = read_in_file_to_df("NCAATourneyCompactResults")
tourney_detailed_results = read_in_file_to_df("NCAATourneyDetailedResults")
reg_season_detailed_results = read_in_file_to_df("RegularSeasonDetailedResults")
reg_season_compact_results = read_in_file_to_df("RegularSeasonCompactResults")
sample_sub = read_in_file_to_df("SampleSubmissionStage1")
seasons = read_in_file_to_df("Seasons")

Teams (364, 4)
NCAATourneyCompactResults (2117, 8)
NCAATourneyDetailedResults (981, 34)
RegularSeasonDetailedResults (76636, 34)
RegularSeasonCompactResults (150684, 8)
SampleSubmissionStage1 (9112, 2)
Seasons (34, 6)


In [6]:
all_detailed_results = pd.concat([reg_season_detailed_results, tourney_detailed_results])
print(all_detailed_results.shape)

(77617, 34)


### Calculate Advanced Metrics

#### Add in possession features

In [7]:
all_detailed_results = create_poss_fields(all_detailed_results)
all_detailed_results = create_ppp_fields(all_detailed_results)

#### Add advanced metrics
Takes a few seconds...

In [64]:
all_detailed_results = add_advanced_metrics(all_detailed_results)

#### Add in team names

In [8]:
print(all_detailed_results.shape)
all_detailed_results = add_in_team_names(all_detailed_results, teams)
print(all_detailed_results.shape)

(77617, 39)
(77617, 41)


#### Add in date from DayNum

In [9]:
print(all_detailed_results.shape)
all_detailed_results = pd.merge(left=all_detailed_results, right=seasons[['Season', 'DayZero']], 
                                left_on='Season', right_on='Season', how='inner')

all_detailed_results['game_date'] = all_detailed_results.apply(lambda row: datetime.datetime.strptime(row['DayZero'],"%m/%d/%Y") + datetime.timedelta(days=row['DayNum']), axis=1)
all_detailed_results = all_detailed_results.drop('DayZero', 1)
print(all_detailed_results.shape)

(77617, 41)
(77617, 42)


## Create train data set

Train data set structure:  
* Result field is binary 0/1 representing if Team1 beats Team2
* Every game is represented by 2 different records

In [12]:
train = create_master_train(all_detailed_results)
print(train.shape)

(155234, 43)


#### Add rolling features

In [14]:
train = create_rolling_avg_feature(train, "t1_PPP", num_games=5)

#### Look at all train records

In [16]:
HTML(train[(train['Season']==2017) & (train['t1_TeamID']==1207)].sort_values('DayNum').head().to_html())

,index,DayNum,NumOT,Result,Season,TotPoss,game_date,t1_Ast,t1_Blk,t1_DR,t1_FGA,t1_FGA3,t1_FGM,t1_FGM3,t1_FTA,t1_FTM,t1_Loc,t1_OR,t1_PF,t1_PPP,t1_Poss,t1_Score,t1_Stl,t1_TO,t1_TeamID,t1_TeamName,t2_Ast,t2_Blk,t2_DR,t2_FGA,t2_FGA3,t2_FGM,t2_FGM3,t2_FTA,t2_FTM,t2_OR,t2_PF,t2_PPP,t2_Poss,t2_Score,t2_Stl,t2_TO,t2_TeamID,t2_TeamName,t1_PPP_rolling_avg_5gm_t1_PPP
44742,16536,12,0,1.0,2017,152.50,2016-11-12,22,2,25,54,17,36,9,32,24,H,8,24,1.377049,76.20,105,13,15,1207,Georgetown,11,2,13,57,25,21,6,28,12,20,26,0.786885,76.30,60,6,26,1367,SC Upstate,NaN
44743,94726,15,0,0.0,2017,148.40,2016-11-15,7,4,24,50,15,16,6,42,37,H,11,24,1.010782,73.95,75,6,15,1207,Georgetown,11,7,25,58,27,26,7,22,17,9,32,1.024259,74.45,76,6,15,1268,Maryland,NaN
44744,94716,17,0,0.0,2017,142.50,2016-11-17,14,3,18,56,20,23,3,32,23,H,12,25,1.010526,70.20,72,5,11,1207,Georgetown,12,2,26,51,12,28,6,28,16,10,29,1.094737,72.30,78,3,18,1117,Arkansas St,NaN
44745,15337,21,0,1.0,2017,141.95,2016-11-21,10,10,26,61,21,21,6,22,17,N,14,21,0.915815,72.45,65,8,15,1207,Georgetown,10,10,30,59,17,23,4,20,11,16,22,0.859458,69.50,61,7,17,1332,Oregon,NaN
44746,94712,22,0,0.0,2017,131.10,2016-11-22,7,1,20,47,15,18,5,20,16,N,1,16,0.869565,65.50,57,6,10,1207,Georgetown,12,5,30,65,22,28,5,16,12,20,18,1.113654,65.60,73,5,13,1458,Wisconsin,NaN


# Modeling

* Build models to predict 0/1 winner - logistic regression
* Build models to predict point spread - secondary model to convert pointspreads to percentages